# DBSCAN
This notebook is split into three sections with one
for every algorithm

## Initialise PySpark and data

In [ ]:
from pyspark import SparkContext, SparkConf, SQLContext

SPARK_CONF = SparkConf()
SPARK_CONF.set("spark.driver.memory", "10g")
SPARK_CONF.set("spark.cores.max", "4")
SPARK_CONF.set("spark.executor.heartbeatInterval", "3600")
SPARK_CONF.setAppName("word2vec")

SPARK_CONTEXT = SparkContext.getOrCreate(SPARK_CONF)
SPARK = SQLContext(SPARK_CONTEXT)

# Load data: TODO

## Scikit

## PyCaret

## Slides
Reading around did